In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBRegressor, plot_importance, plot_tree
from tqdm import tqdm_notebook
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import math
import datetime as dt
import joblib

%matplotlib qt5

In [ ]:
def get_mov_avg_std(df, col, N):
    """
    Given a dataframe, get mean and std dev at timestep t using values from t-1, t-2, ..., t-N.
    Inputs
        df         : dataframe. Can be of any length.
        col        : name of the column you want to calculate mean and std dev
        N          : get mean and std dev at timestep t using values from t-1, t-2, ..., t-N
    Outputs
        df_out     : same as df but with additional column containing mean and std dev
    """

    mean_list = df[col].rolling(window = N, min_periods=1).mean() # len(mean_list) = len(df)
    std_list = df[col].rolling(window = N, min_periods=1).std()   # first value will be NaN, because normalized by N-1
    
    # Add one timestep to the predictions
    mean_list = np.concatenate((np.array([np.nan]), np.array(mean_list[:-1])))
    std_list = np.concatenate((np.array([np.nan]), np.array(std_list[:-1])))
    
    # Append mean_list to df
    df_out = df.copy()
    df_out[col + '_mean'] = mean_list
    df_out[col + '_std'] = std_list
    
    return df_out

def scale_row(row, feat_mean, feat_std):
    """
    Given a pandas series in row, scale it to have 0 mean and var 1 using feat_mean and feat_std
    Inputs
        row      : pandas series. Need to scale this.
        feat_mean: mean  
        feat_std : standard deviation
    Outputs
        row_scaled : pandas series with same length as row, but scaled
    """
    # If feat_std = 0 (this happens if adj_close doesn't change over N days), 
    # set it to a small number to avoid division by zero
    feat_std = 0.001 if feat_std == 0 else feat_std
    
    row_scaled = (row-feat_mean) / feat_std
    
    return row_scaled

In [ ]:
def get_mov_avg_std(df, col, N):
    """
    Given a dataframe, get mean and std dev at timestep t using values from t-1, t-2, ..., t-N.
    Inputs
        df         : dataframe. Can be of any length.
        col        : name of the column you want to calculate mean and std dev
        N          : get mean and std dev at timestep t using values from t-1, t-2, ..., t-N
    Outputs
        df_out     : same as df but with additional column containing mean and std dev
    """
    mean_list = df[col].rolling(window = N, min_periods=1).mean() # len(mean_list) = len(df)
    std_list = df[col].rolling(window = N, min_periods=1).std()   # first value will be NaN, because normalized by N-1
    
    # Add one timestep to the predictions
    mean_list = np.concatenate((np.array([np.nan]), np.array(mean_list[:-1])))
    std_list = np.concatenate((np.array([np.nan]), np.array(std_list[:-1])))
    
    # Append mean_list to df
    df_out = df.copy()
    df_out[col + '_mean'] = mean_list
    df_out[col + '_std'] = std_list
    
    return df_out

def do_scaling(df, N, cols_list):
    df_scaled = df.copy()
    
    for col in tqdm_notebook(cols_list):
        feat_list = [col + '_lag_' + str(shift) for shift in range(1, N+1)]
        save_list = [col + '_scaled_lag_' + str(shift) for shift in range(1, N+1)]
        temp = df.apply(lambda row: scale_row(row[feat_list], row[col+'_mean'], row[col+'_std']), axis=1)
        temp.rename(columns={feat_list[i]:save_list[i] for i in range(len(save_list))}, inplace=True)
        
        df_scaled = pd.concat([df_scaled, temp], axis=1)

    return df_scaled

def pred_xgboost(model, N, H, prev_vals, prev_mean_val, prev_std_val, lag_cols):
    """
    Do recursive forecasting using xgboost
    Inputs
        model              : the xgboost model
        N                  : for feature at day t, we use lags from t-1, t-2, ..., t-N as features
        H                  : forecast horizon
        prev_vals          : numpy array. If predict at time t, 
                             prev_vals will contain the N unscaled values at t-1, t-2, ..., t-N
        prev_mean_val      : the mean of the unscaled values at t-1, t-2, ..., t-N
        prev_std_val       : the std deviation of the unscaled values at t-1, t-2, ..., t-N
    Outputs
        Times series of predictions. Numpy array of shape (H,). This is unscaled.
    """
    forecast = prev_vals.copy()
    
    for n in range(H):
        filter_date = forecast['Date'].max() - dt.timedelta(days=N)
        forecast = forecast[forecast['Date'] >= filter_date]
        
             
        # Do prediction
        est_scaled = model.predict(forecast)
        
        # Precisa fazer: 
        # - Combinar resultado da predição com os dados para proxima predição
        # - Coletar informações adicionais e rodar novamente lag cols 
        # - Criar um array de resposta 

           
    return forecast[-H:]

def train_pred_eval_model(X_train_scaled,
                          y_train_scaled,
                          y_test,
                          N,
                          H,
                          prev_vals,
                          prev_mean_val,
                          prev_std_val,
                          lag_cols,
                          seed=100,
                          n_estimators=100,
                          max_depth=3,
                          learning_rate=0.1,
                          min_child_weight=1,
                          subsample=1,
                          colsample_bytree=1,
                          colsample_bylevel=1,
                          gamma=0):
    '''
    Train model, do prediction, scale back to original range and do evaluation
    Use XGBoost here.
    Inputs
        X_train_scaled     : features for training. Scaled to have mean 0 and variance 1
        y_train_scaled     : target for training. Scaled to have mean 0 and variance 1
        y_test             : target for test. Actual values, not scaled.
        N                  : for feature at day t, we use lags from t-1, t-2, ..., t-N as features
        H                  : forecast horizon
        prev_vals          : numpy array. If scaled[0] is at time t, prev_vals will contain the N-1 unscaled values at t-1, t-2, ...
        prev_mean_val      : the mean of the unscaled values at t-1, t-2, ..., t-N
        prev_std_val       : the std deviation of the unscaled values at t-1, t-2, ..., t-N
        seed               : model seed
        n_estimators       : number of boosted trees to fit
        max_depth          : maximum tree depth for base learners
        learning_rate      : boosting learning rate (xgb’s “eta”)
        min_child_weight   : minimum sum of instance weight(hessian) needed in a child
        subsample          : subsample ratio of the training instance
        colsample_bytree   : subsample ratio of columns when constructing each tree
        colsample_bylevel  : subsample ratio of columns for each split, in each level
        gamma              : 
    Outputs
        rmse               : root mean square error of y_test and est
        mape               : mean absolute percentage error of y_test and est
        mae                : mean absolute error of y_test and est
        est                : predicted values. Same length as y_test
    '''

    model = XGBRegressor(objective ='reg:squarederror',
                         seed=seed,
                         n_estimators=n_estimators,
                         max_depth=max_depth,
                         learning_rate=learning_rate,
                         min_child_weight=min_child_weight,
                         subsample=subsample,
                         colsample_bytree=colsample_bytree,
                         colsample_bylevel=colsample_bylevel,
                         gamma=gamma)
    
    # Train the model
    model.fit(X_train_scaled, y_train_scaled)
    
    # Get predicted labels and scale back to original range
    est = pred_xgboost(model, N, H, prev_vals, prev_mean_val, prev_std_val, lag_cols)

    # Calculate RMSE, MAPE, MAE
#     rmse = get_rmse(y_test, est)
#     mape = get_mape(y_test, est)
#     mae = get_mae(y_test, est)
    
#     return rmse, mape, mae, est, model.feature_importances_

def add_lags(df, N, lag_cols):
    """
    Add lags up to N number of days to use as features
    The lag columns are labelled as 'adj_close_lag_1', 'adj_close_lag_2', ... etc.
    """
    # Use lags up to N number of days to use as features
    df_w_lags = df.copy()
    
    for country in df_w_lags['Country'].unique():
        df_w_lags.loc[df_w_lags['Country'] == country, 'order_day'] = [x for x in list(range(len(df_w_lags.loc[df_w_lags['Country'] == country, :])))]

    # merging_keys
    merging_keys = ['Country', 'country_code','order_day']
    
    shift_range = [x+1 for x in range(N)]
    
    for shift in tqdm_notebook(shift_range):
        train_shift = df_w_lags[merging_keys + lag_cols].copy()
    
        for country in df_w_lags['Country'].unique():    
            # E.g. order_day of 0 becomes 1, for shift = 1.
            # So when this is merged with order_day of 1 in df_confirmed, this will represent lag of 1.

            train_shift.loc[train_shift['Country'] == country, 'order_day'] =train_shift.loc[train_shift['Country'] == country, 'order_day'] + shift

        foo = lambda x: '{}_lag_{}'.format(x, shift) if x in lag_cols else x
        train_shift = train_shift.rename(columns=foo)

        df_w_lags = pd.merge(df_w_lags, train_shift, on=merging_keys, how='left')
    
    del train_shift
    
    return df_w_lags

def get_error_metrics(df,
                      val_date,
                      N,
                      H,
                      lag_cols,
                      target_col,
                      seed=100,
                      n_estimators=100,
                      max_depth=3,
                      learning_rate=0.1,
                      min_child_weight=1,
                      subsample=1,
                      colsample_bytree=1,
                      colsample_bylevel=1,
                      gamma=0):
    """
    Given a series consisting of both train+validation, do predictions of forecast horizon H on the validation set, 
    at H/2 intervals.
    Inputs
        df                 : train + val dataframe. len(df) = train_size + val_size
        train_size         : size of train set
        N                  : for feature at day t, we use lags from t-1, t-2, ..., t-N as features
        H                  : forecast horizon
        seed               : model seed
        n_estimators       : number of boosted trees to fit
        max_depth          : maximum tree depth for base learners
        learning_rate      : boosting learning rate (xgb’s “eta”)
        min_child_weight   : minimum sum of instance weight(hessian) needed in a child
        subsample          : subsample ratio of the training instance
        colsample_bytree   : subsample ratio of columns when constructing each tree
        colsample_bylevel  : subsample ratio of columns for each split, in each level
        gamma              : 

    Outputs
        mean of rmse, mean of mape, mean of mae, dictionary of predictions
    """
    rmse_list = [] # root mean square error
    mape_list = [] # mean absolute percentage error
    mae_list = []  # mean absolute error
    preds_dict = {}
    
    original_cols = df.columns.values
    # Add lags up to N number of days to use as features
    df = add_lags(df, N, lag_cols)
    
    # Get mean and std dev at timestamp t using values from t-1, ..., t-N
    for col in lag_cols:
        df = get_mov_avg_std(df, col, N)
    
    # Do scaling
    df = do_scaling(df, N, lag_cols)
        
    # Get list of features
    features =  ['country_code']
    for n in range(N,0,-1):
        for col in lag_cols:
            features.append(col+"_scaled_lag_"+str(n))
    
    
    # Split into train and test
    train = df[df['Date'] < val_date].copy()
    test = df[df['Date'] >= val_date].copy()
    
    train.sort_values(by=['Country', 'Date'], inplace=True)
    test.sort_values(by=['Country', 'Date'], inplace=True)
        
    # Split into X and y
    X_train_scaled = train[features]
    y_train_scaled = train[target_col]
    y_test = test[target_col]
    mask_prev_vals = (train['Date'] >= (val_date - dt.timedelta(days=N)))
    prev_vals = (train.loc[mask_prev_vals, features])
    prev_mean_val = test.iloc[0][target_col+'_mean']
    prev_std_val = test.iloc[0][target_col+'_std']
    
    print("X_train_scaled.shape:", X_train_scaled.shape)
    print("y_train_scaled.shape", y_train_scaled.shape)
    print("y_test_scaled.shape:", y_test.shape)
    print("prev_vals.shape:", prev_vals.shape)
    print(prev_mean_val)
    print(prev_std_val)
    
    # Train and predict D+N
    rmse, mape, mae, est, _ = train_pred_eval_model(X_train_scaled,
                                                    y_train_scaled,
                                                    y_test,
                                                    N,
                                                    H,
                                                    prev_vals,
                                                    prev_mean_val,
                                                    prev_std_val,
                                                    lag_cols,
                                                    seed=seed,
                                                    n_estimators=n_estimators,
                                                    max_depth=max_depth,
                                                    learning_rate=learning_rate,
                                                    min_child_weight=min_child_weight,
                                                    subsample=subsample,
                                                    colsample_bytree=colsample_bytree,
                                                    colsample_bylevel=colsample_bylevel,
                                                    gamma=gamma)
    
    return rmse, mape, mae, est 

# Leitura dos dados

In [ ]:
df_covid = pd.read_csv("./input/covid_19_clear.csv", parse_dates=['Date'], infer_datetime_format=True)

In [ ]:
df_covid.info()

In [ ]:
ax = df_covid[df_covid['Country'] == 'Brazil'].plot(x='Date', y='Confirmed', style='b-', grid=True)
ax.set_xlabel("date")
ax.set_ylabel("Confirmed")

In [ ]:
df_info = pd.read_csv("./input/country_info.csv", parse_dates=['Lockdown Start Date'], infer_datetime_format=True)

In [ ]:
df_info.info()

In [ ]:
df = pd.merge(df_covid, df_info, left_on='Country', right_on='Country', how='inner', suffixes=('', ''))

In [ ]:
del df_covid
del df_info

In [ ]:
df.loc[df['Date'].dt.month == 1, 'Country Temperature ºC'] = df['Temperature Jan (ºC)']
df.loc[df['Date'].dt.month == 2, 'Country Temperature ºC'] = df['Temperature Feb (ºC)']
df.loc[df['Date'].dt.month == 3, 'Country Temperature ºC'] = df['Temperature Mar (ºC)']
df.loc[df['Date'].dt.month == 4, 'Country Temperature ºC'] = df['Temperature Apr (ºC)']

In [ ]:
n_cases = 50

df = df[df['Confirmed'] > n_cases]

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
df["country_code"] = lb_make.fit_transform(df["Country"])

In [ ]:
df.sort_values(by=['Country','Date'], inplace=True)

# Separação dos conjuntos de treinamento para casos confirmados e mortes

In [ ]:
df_confirmed = df.drop(columns=['Temperature Jan (ºC)', 'Temperature Feb (ºC)', 'Temperature Mar (ºC)', 'Temperature Apr (ºC)', 'Deaths', 'Recovered', 'Daily new confirmed deaths due to COVID-19 (rolling 3-day average)', 'Hospital beds (per 1,000 people)', 'Lockdown Start Date'])
df_deaths = df.drop(columns=['Temperature Jan (ºC)', 'Temperature Feb (ºC)', 'Temperature Mar (ºC)', 'Temperature Apr (ºC)', 'Recovered', 'Lockdown Start Date'])

# Treinamento e Predição

## Determina o numero de dias que quer prever (horizonte)

In [ ]:
H = 10

valid_date = df_confirmed.Date.max() - dt.timedelta(days=H)

print("Predicting on date %s, with forecast horizon H = %d" % (valid_date, H))

# Treina e prevê os valores futuros para confirmados

In [ ]:
N = 4 # Numeros de dias para usar como lag

lag_cols = df_confirmed.columns.values.tolist()[2:22] # Colunas para gerar lag
target_col = 'Confirmed'

get_error_metrics(df_confirmed, valid_date, N, H, lag_cols, target_col)